-----

## Setup

In [6]:
use Data::Importers;
use Markdown::Grammar;

use EBNF::Grammar;
use FunctionalParsers :ALL;
use FunctionalParsers::EBNF;
use Grammar::TokenProcessing;

use JSON::Fast;
use LLM::Configurations;

-----

## Translations

In [17]:
my $ebnfCode1 = q:to/END/;
<top> = <who> , <verb> , <lang> ;
<who> = 'I' | 'We' ;
<verb> = ( 'love' | 'hate' | 'like' | { '❤️' } | '🤮' ) <@ MyVerb ;
<lang> = ( 'Julia' | 'Perl' | 'Python' | 'R' | 'Raku' | 'WL' ) <@ MyLang; 
END

fp-ebnf-parse($ebnfCode1, target=>"WL::Code", dir-spec => 'LR').head.tail.join("\n")

pTOP = ParseSequentialComposition[pWHO, ParseSequentialComposition[pVERB, pLANG]];
pWHO = ParseAlternativeComposition[ParseSymbol["I"], ParseSymbol["We"]];
pVERB = ParseApply[MyVerb, ParseAlternativeComposition[ParseSymbol["love"], ParseSymbol["hate"], ParseSymbol["like"], ParseMany[ParseSymbol["❤️"]], ParseSymbol["🤮"]]];
pLANG = ParseApply[MyLang, ParseAlternativeComposition[ParseSymbol["Julia"], ParseSymbol["Perl"], ParseSymbol["Python"], ParseSymbol["R"], ParseSymbol["Raku"], ParseSymbol["WL"]]];

-----

## Ingest Markdown dictionary

In [21]:
my $fileName = $*CWD ~ '/Functional-Parsers-WL-EBNF.md';
my $mtext = slurp($fileName);

text-stats($mtext)

(chars => 1346 words => 151 lines => 41)

In [22]:
my @tree = md-section-tree($mtext);

[Functional Parsers (WL <-> EBNF) => [(Whatever) => () 1 => ({content => 
, level => 7, name => (Whatever), type => md-text-block} {content => ```ebnf
<top> = <a> | <b> ;
<a> = 'a' , { 'A' } , [ '1' ];
<b> = 'b' , ( 'B' | '2' );
```, level => 7, name => (Whatever), type => md-code-block} {content => 

, level => 7, name => (Whatever), type => md-text-block} {content => ```mathematica
pTOP = ParseAlternativeComposition[pA, pB]; 
pA = ParseSequentialComposition[ParseSymbol["a"], ParseSequentialComposition[ParseMany[ParseSymbol["A"]], ParseOption[ParseSymbol["1"]]]]; 
pB = ParseSequentialComposition[ParseSymbol["b"], ParseAlternativeComposition[ParseSymbol["B"], ParseSymbol["2"]]];
```, level => 7, name => (Whatever), type => md-code-block} {content => 

, level => 7, name => (Whatever), type => md-text-block}) 2 => ({content => 
, level => 7, name => (Whatever), type => md-text-block} {content => ```ebnf
<args> = <newArg> | <oldArg> <@ MyFunc;
```, level => 7, name => (Whatever), type =>

In [23]:
@tree.head.value

[(Whatever) => () 1 => ({content => 
, level => 7, name => (Whatever), type => md-text-block} {content => ```ebnf
<top> = <a> | <b> ;
<a> = 'a' , { 'A' } , [ '1' ];
<b> = 'b' , ( 'B' | '2' );
```, level => 7, name => (Whatever), type => md-code-block} {content => 

, level => 7, name => (Whatever), type => md-text-block} {content => ```mathematica
pTOP = ParseAlternativeComposition[pA, pB]; 
pA = ParseSequentialComposition[ParseSymbol["a"], ParseSequentialComposition[ParseMany[ParseSymbol["A"]], ParseOption[ParseSymbol["1"]]]]; 
pB = ParseSequentialComposition[ParseSymbol["b"], ParseAlternativeComposition[ParseSymbol["B"], ParseSymbol["2"]]];
```, level => 7, name => (Whatever), type => md-code-block} {content => 

, level => 7, name => (Whatever), type => md-text-block}) 2 => ({content => 
, level => 7, name => (Whatever), type => md-text-block} {content => ```ebnf
<args> = <newArg> | <oldArg> <@ MyFunc;
```, level => 7, name => (Whatever), type => md-code-block} {content => 

, level

In [24]:
#% html
@tree.head.values ==> to-html

(Whatever) 1 type name content level md-text-block (Whatever) 
 7 md-code-block (Whatever) ```ebnf
<top> = <a> | <b> ;
<a> = 'a' , { 'A' } , [ '1' ];
<b> = 'b' , ( 'B' | '2' );
``` 7 md-text-block (Whatever) 

 7 md-code-block (Whatever) ```mathematica
pTOP = ParseAlternativeComposition[pA, pB]; 
pA = ParseSequentialComposition[ParseSymbol["a"], ParseSequentialComposition[ParseMany[ParseSymbol["A"]], ParseOption[ParseSymbol["1"]]]]; 
pB = ParseSequentialComposition[ParseSymbol["b"], ParseAlternativeComposition[ParseSymbol["B"], ParseSymbol["2"]]];
``` 7 md-text-block (Whatever) 

 7 2 name content level type (Whatever) 
 7 md-text-block (Whatever) ```ebnf
<args> = <newArg> | <oldArg> <@ MyFunc;
``` 7 md-code-block (Whatever) 

 7 md-text-block (Whatever) ```mathematica
pARGS = ParseAlternativeComposition[pNEWARG, ParseApply[MyFunc, pOLDARG]];
``` 7 md-code-block (Whatever) 

 7 md-text-block 3 name level type content (Whatever) 7 md-text-block 
 (Whatever) 7 md-code-block ```ebnf
<top> = <who> , <verb> , <lang> ;
<who> = 'I' | 'We' ;
<verb> = ( 'love' | 'hate' | 'like' | { '❤️' } | '🤮' ) <@ MyVerb ;
<lang> = ( 'Julia' | 'Perl' | 'Python' | 'R' | 'Raku' | 'WL' ) <@ MyLang; 
``` (Whatever) 7 md-text-block 

 (Whatever) 7 md-code-block ```mathematica
pTOP = ParseSequentialComposition[pWHO, ParseSequentialComposition[pVERB, pLANG]];
pWHO = ParseAlternativeComposition[ParseSymbol["I"], ParseSymbol["We"]];
pVERB = ParseApply[MyVerb, ParseAlternativeComposition[ParseSymbol["love"], ParseSymbol["hate"], ParseSymbol["like"], ParseMany[ParseSymbol["❤️"]], ParseSymbol["🤮"]]];
pLANG = ParseApply[MyLang, ParseAlternativeComposition[ParseSymbol["Julia"], ParseSymbol["Perl"], ParseSymbol["Python"], ParseSymbol["R"], ParseSymbol["Raku"], ParseSymbol["WL"]]];
``` (Whatever) 7 md-text-block

In [26]:
my @dsCode = @tree.head.value.grep({ $_.key.defined && $_.value.defined && $_.elems }).map({ $_.key => $_.value.grep({ $_.<content>.trim.chars }).Array });;

deduce-type(@dsCode):tally

Tuple([Pair(Atom((Str)), Vector(Struct([content, level, name, type], [Str, Int, Whatever, Str]), 2)) => 3], 3)

In [27]:
my %genRules = @dsCode.map({ $_.value.tail<content> => $_.value.head<content> })

{```mathematica
pARGS = ParseAlternativeComposition[pNEWARG, ParseApply[MyFunc, pOLDARG]];
``` => ```ebnf
<args> = <newArg> | <oldArg> <@ MyFunc;
```, ```mathematica
pTOP = ParseAlternativeComposition[pA, pB]; 
pA = ParseSequentialComposition[ParseSymbol["a"], ParseSequentialComposition[ParseMany[ParseSymbol["A"]], ParseOption[ParseSymbol["1"]]]]; 
pB = ParseSequentialComposition[ParseSymbol["b"], ParseAlternativeComposition[ParseSymbol["B"], ParseSymbol["2"]]];
``` => ```ebnf
<top> = <a> | <b> ;
<a> = 'a' , { 'A' } , [ '1' ];
<b> = 'b' , ( 'B' | '2' );
```, ```mathematica
pTOP = ParseSequentialComposition[pWHO, ParseSequentialComposition[pVERB, pLANG]];
pWHO = ParseAlternativeComposition[ParseSymbol["I"], ParseSymbol["We"]];
pVERB = ParseApply[MyVerb, ParseAlternativeComposition[ParseSymbol["love"], ParseSymbol["hate"], ParseSymbol["like"], ParseMany[ParseSymbol["❤️"]], ParseSymbol["🤮"]]];
pLANG = ParseApply[MyLang, ParseAlternativeComposition[ParseSymbol["Julia"], ParseSymbol["Perl"]

In [28]:
my $prompt = q:to/END/;
You are a translator of natural language specifications or Extended Backus Naur Form (EBNF) specifications of formal grammars into Mathematica code.

Here are example translations:
END

$prompt = $prompt ~ %genRules.map({ "\n\nInput:\n{$_.key}\n\nOutput:\n{$_.value}"});

You are a translator of natural language specifications or Extended Backus Naur Form (EBNF) specifications of formal grammars into Mathematica code.

Here are example translations:


Input:
```mathematica
pTOP = ParseAlternativeComposition[pA, pB]; 
pA = ParseSequentialComposition[ParseSymbol["a"], ParseSequentialComposition[ParseMany[ParseSymbol["A"]], ParseOption[ParseSymbol["1"]]]]; 
pB = ParseSequentialComposition[ParseSymbol["b"], ParseAlternativeComposition[ParseSymbol["B"], ParseSymbol["2"]]];
```

Output:
```ebnf
<top> = <a> | <b> ;
<a> = 'a' , { 'A' } , [ '1' ];
<b> = 'b' , ( 'B' | '2' );
``` 

Input:
```mathematica
pARGS = ParseAlternativeComposition[pNEWARG, ParseApply[MyFunc, pOLDARG]];
```

Output:
```ebnf
<args> = <newArg> | <oldArg> <@ MyFunc;
``` 

Input:
```mathematica
pTOP = ParseSequentialComposition[pWHO, ParseSequentialComposition[pVERB, pLANG]];
pWHO = ParseAlternativeComposition[ParseSymbol["I"], ParseSymbol["We"]];
pVERB = ParseApply[MyVerb, ParseAlternativeComp

-----

## Experiments

### Complicated prompt

In [29]:
my $res = llm-synthesize([
    $prompt,
    "Translate the formal EBNF grammar for parsing arithmetic expressions.",
    llm-prompt('NothingElse')('Mathematica')
    ],
    e => $conf4o
)

```mathematica
pEXPR = ParseAlternativeComposition[pTERM, ParseSequentialComposition[ParseSymbol["+"], pEXPR]];
pTERM = ParseAlternativeComposition[pFACTOR, ParseSequentialComposition[ParseSymbol["*"], pTERM]];
pFACTOR = ParseAlternativeComposition[ParseSymbol["("], ParseSequentialComposition[pEXPR, ParseSymbol[")"]], ParseSymbol["number"]];
```